In [1]:
import numpy as np

import seaborn as sns
import pandas as pd
import os
import pprint
import ast
import re
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import json

pp = pprint.PrettyPrinter(depth=6)

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})


<Response [200]>

In [2]:
# get a set of all monomers with an associated uniprot id
proteins_df = pd.read_csv('reconstruction/ecoli/flat/proteins.tsv', sep='\t').loc[:, ["id", "common_name"]]

for column in ["enzyme_reaction", "cofactors", "metal_features", "other_features"]:
    proteins_df[column] = 0
    proteins_df[column] = proteins_df[column].astype(object)

proteins_df

,id,common_name,enzyme_reaction,cofactors,metal_features,other_features
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,1-acylglycerol-3-phosphate <i>O</i>-acyltransf...,0,0,0,0
1,1-PFK-MONOMER,1-phosphofructokinase,0,0,0,0
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2-dehydropantoate 2-reductase,0,0,0,0
3,2-ISOPROPYLMALATESYN-MONOMER,2-isopropylmalate synthase,0,0,0,0
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,"bifunctional 2-octaprenyl-6-methoxy-1,4-benzoq...",0,0,0,0
...,...,...,...,...,...,...
4415,YTFR-MONOMER,galactofuranose ABC transporter putative ATP b...,0,0,0,0
4416,YTFT-MONOMER,galactofuranose ABC transporter putative membr...,0,0,0,0
4417,ZNUA-MONOMER,Zn<sup>2+</sup> ABC transporter periplasmic bi...,0,0,0,0
4418,ZNUB-MONOMER,Zn<sup>2+</sup> ABC transporter membrane subunit,0,0,0,0


## Get relevant EcoCyc data and annotate protein monomer table

In [3]:
# example entry
protein = 'EG11333-MONOMER'
req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{protein}&detail=high'

r = s.get(req_str)
if r.status_code != 200:
    print(protein, r.status_code)

o = xmltodict.parse(r.content)['ptools-xml']
pp.pprint(o)

{'@ptools-version': '27.0',
 '@xml:base': 'http://BioCyc.org/getxml?ECOLI:EG11333-MONOMER',
 'Protein': {'@ID': 'ECOLI:EG11333-MONOMER',
             '@detail': 'full',
             '@frameid': 'EG11333-MONOMER',
             '@orgid': 'ECOLI',
             'catalyzes': {'Enzymatic-Reaction': {'@ID': 'ECOLI:ENZRXN0-7774',
                                                  '@detail': 'low',
                                                  '@frameid': 'ENZRXN0-7774',
                                                  '@orgid': 'ECOLI',
                                                  'common-name': {'#text': '2-octaprenylphenol '
                                                                           '6-hydroxylase',
                                                                  '@datatype': 'string'},
                                                  'enzyme': {'Protein': {'@frameid': 'EG11333-MONOMER',
                                                                         '@org

In [ ]:
for i in range(len(proteins_df.index)):
    if i % 100 == 0:
        print(i)

    protein = proteins_df.loc[i, 'id']
    proteins_df.at[i, 'other_features'] = set()
    proteins_df.at[i, 'metal_features'] = set()
    proteins_df.at[i, 'enzyme_reaction'] = set()
    proteins_df.at[i, 'cofactors'] = set()

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{protein}&detail=high'



    r = s.get(req_str)
    if r.status_code != 200:
        print(protein, r.status_code)
        continue

    o = xmltodict.parse(r.content)['ptools-xml']

    metal_set = set()
    other_feature_set = set()
    if 'Protein' in o and 'has-feature' in o['Protein']:
        features = o['Protein']['has-feature']

        if type(features) is dict:
            features = [features]

        for feature in features:
            if 'parent' not in feature['Feature']:
                continue

            category = feature['Feature']['parent']['Feature']['@frameid']
            if category == 'Metal-Binding-Sites' and 'comment' in feature['Feature']:

                # Detect match to any of the allowed metal names and allowed cofactor names and add to list
                comment = feature['Feature']['comment']['#text']
                metal_set.add(comment)

                proteins_df.at[i, 'metal_features'] = list(metal_set)

            if category == 'Nucleotide-Phosphate-Binding-Regions' and 'attached-group' in feature['Feature'] and 'Compound' in feature['Feature']['attached-group']:
                attached_group = feature['Feature']['attached-group']['Compound']['@frameid']
                other_feature_set.add(attached_group)

            if category == 'N6-pyridoxal-phosphate-Lys-Modifications':
                other_feature_set.add('PYRIDOXAL_PHOSPHATE')

            if category == 'Protein-Segments' and 'comment' in feature['Feature'] and 'Thiamine' in feature['Feature']['comment']['#text']:
                other_feature_set.add('THIAMINE-PYROPHOSPHATE')

            if category == 'Selenocysteine-site':
                metal_set.add('L-SELENOCYSTEINE')

            proteins_df.at[i, 'metal_features'] = list(metal_set)
            proteins_df.at[i, 'other_features'] = list(other_feature_set)

        if 'catalyzes' in o['Protein']:
            oc = o['Protein']['catalyzes']['Enzymatic-Reaction']

            if type(oc) is dict:
                oc = [oc]

            cofactor_set = set()
            enz_rxn_set = set()

            for enzrxn in oc:
                enz_id = enzrxn['@frameid']

                enz_rxn_set.add(enz_id)

                enz_req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_id}&detail=high'

                rz = s.get(enz_req_str)
                oe = xmltodict.parse(rz.content)['ptools-xml']['Enzymatic-Reaction']

                if "cofactor" in oe:
                    oe = oe['cofactor']

                    if type(oe) is dict:
                        oe = [oe]

                    for cofactor in oe:
                        cof = cofactor['Compound']['@frameid']
                        cofactor_set.add(cof)

            proteins_df.at[i, 'enzyme_reaction'] = enz_rxn_set
            proteins_df.at[i, 'cofactors'] = cofactor_set




0


In [ ]:
proteins_df

In [ ]:
# proteins_df.to_parquet('notebooks/fbagd/data/raw_protein_features.parquet', index=False)

## Reload protein feature table

In [ ]:
# proteins_df = pd.read_parquet('notebooks/fbagd/data/raw_protein_features.parquet')
# proteins_df['metal_features'] = proteins_df['metal_features'].apply(ast.literal_eval)
# # for rows of proteins with where other_features is set, convert from string to set with literal_eval
# proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'] = \
#     proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'].apply(ast.literal_eval)

filter_protein_df = proteins_df.copy().loc[:, ['id', 'common_name', 'metal_features', 'other_features', 'enzyme_reaction', 'cofactors']]
filter_protein_df

## Process raw EcoCyc annotations into standard EcoCyc names

In [ ]:
ALLOWED_METAL_NAMES =   {'Iron': 'FE+2', 'Cobalt': 'CO+2', 'Copper': 'CU+2', 'Manganese': 'MN+2', 'Molybdenum': 'CPD-8123', 'Nickel': 'NI+2', 'Tungsten': 'W', 'Zinc': 'ZN+2',
                        'Calcium': 'CA+2', 'Magnesium': 'MG+2', 'Sodium': 'NA+', 'Potassium': 'K+',
                        'Iron-sulfur \(4Fe-4S\)': 'CPD-7', 'Iron-sulfur \(2Fe-2S\)': 'CPD-6',
                         'Iron-sulfur \(4Fe-4S-S-AdoMet\)': 'CPD-7', 'Iron-sulfur \(3Fe-4S\)': '3FE-4S', 'Iron-oxo-sulfur \(4Fe-2O-2S\)': 'CPD-7',
                        'heme': 'Heme-b', 'Molybdate': 'CPD-3', 'heme B': 'Heme-b',
                         'L-SELENOCYSTEINE': 'L-SELENOCYSTEINE',
                        'Divalent metal cation': 'Any+2'}

# remove all \ characters from keys in ALLOWED_METAL_NAMES
NON_REGEX_METAL = {key.replace('\\', ''): value for key, value in ALLOWED_METAL_NAMES.items()}

filter_protein_df['metal_features_processed'] = 0
filter_protein_df['metal_features_processed'] = filter_protein_df['metal_features_processed'].astype(object)

metal_pattern = '|'.join(ALLOWED_METAL_NAMES.keys())
metal_regex = re.compile(f'(({metal_pattern})(\s\d[\.,;]|[\.,;]|\s\())')


for i in range(len(filter_protein_df.index)):

    metal_binding = filter_protein_df.loc[i, 'metal_features']

    metal_count_dict = {}
    existing_matches = set()

    for feature in metal_binding:
        matches = metal_regex.search(feature)
        if matches:
            metal = matches.group(0)[:-1]

            # eliminate duplicates
            if metal not in existing_matches:

                existing_matches.add(metal)

                if 'heme' in feature:
                    metal = metal.replace('Iron', 'heme')

                # check if last char of metal is a number, then crop
                if metal[-1].isdigit():
                    metal = metal[:-2]

                metal = metal.strip()

                # replace metal name with allowed metal name
                metal = NON_REGEX_METAL[metal]

                if metal in metal_count_dict:
                    metal_count_dict[metal] += 1
                else:
                    metal_count_dict[metal] = 1

        else:
            print(f'No match for {feature} in {filter_protein_df.loc[i, "id"]}')




    filter_protein_df.at[i, 'metal_features_processed'] = metal_count_dict

In [ ]:
filter_protein_df = filter_protein_df.drop(columns=['metal_features'])
filter_protein_df

In [ ]:
filter_protein_df['other_features_processed'] = 0
filter_protein_df['other_features_processed'] = filter_protein_df['other_features_processed'].astype(object)

ACCEPTED_OTHER_FEATURES = {'PYRIDOXAL_PHOSPHATE', 'THIAMINE-PYROPHOSPHATE', 'FMN', 'FAD'}

for i in range(len(filter_protein_df.index)):

    other_features = filter_protein_df.loc[i, 'other_features']

    other_feature_count_dict = {}
    existing_matches = set()

    for feature in other_features:

        # eliminate duplicates
        if feature not in existing_matches:

            existing_matches.add(feature)

            if feature in ACCEPTED_OTHER_FEATURES:
                if feature in other_feature_count_dict:
                    other_feature_count_dict[feature] += 1
                else:
                    other_feature_count_dict[feature] = 1

    filter_protein_df.at[i, 'other_features_processed'] = other_feature_count_dict

In [ ]:
filter_protein_df = filter_protein_df.drop(columns=['other_features'])

filter_protein_df

In [ ]:
# filter_protein_df.to_parquet('notebooks/fbagd/data/processed_proteins.parquet', index=False)

# Annotate complexation with EcoCyc data

In [ ]:
complex_df = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions.tsv', sep='\t').loc[:, ['id', 'stoichiometry']]


removed_complexes = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions_removed.tsv', sep='\t')

# remove rows where id starts with '#'
complex_df = complex_df[~complex_df['id'].str.startswith('#')].reset_index(drop=True)

# remove rows of complex_df where id matches an id in removed_complexes
complex_df = complex_df[~complex_df['id'].isin(removed_complexes['id'])].reset_index(drop=True)
complex_df.stoichiometry = complex_df.stoichiometry.astype(object)


for i, stoich in enumerate(complex_df.loc[:, 'stoichiometry']):

    if type(stoich) is str and stoich[0] == '{':
        stoich = stoich.replace('null', '-1')
        stoich = ast.literal_eval(stoich)

        complex_df.at[i, 'stoichiometry'] = stoich

    else:
        complex_df.at[i, 'stoichiometry'] = {}


# for each row, find dict entry with positive value
for i in range(len(complex_df.index)):

    stoich = complex_df.loc[i, 'stoichiometry']

    for k,v in stoich.items():
        if v > 0:
            complex_df.at[i, 'id'] = k

complex_df

In [ ]:
complex_df["cofactors"] = 0
complex_df["cofactors"] = complex_df["cofactors"].astype(object)

complex_df["enzyme_reaction"] = 0
complex_df["enzyme_reaction"] = complex_df["enzyme_reaction"].astype(object)

for i in range(len(complex_df.index)):

    if i % 100 == 0:
        print(i)

    complex = complex_df.loc[i, 'id']

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{complex}&detail=low'

    r = s.get(req_str)
    if r.status_code != 200:
        print(complex, r.status_code)
        complex_df.at[i, 'enzyme_reaction'] = set()
        complex_df.at[i, 'cofactors'] = set()
        continue

    o = xmltodict.parse(r.content)['ptools-xml']['Protein']


    # if enzyme
    if 'catalyzes' in o:
        o = o['catalyzes']['Enzymatic-Reaction']

        if type(o) is dict:
            o = [o]

        cofactor_set = set()
        enz_rxn_set = set()

        for enzrxn in o:
            enz_id = enzrxn['@frameid']

            enz_rxn_set.add(enz_id)

            enz_req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_id}&detail=high'

            rz = s.get(enz_req_str)
            oe = xmltodict.parse(rz.content)['ptools-xml']['Enzymatic-Reaction']

            if "cofactor" in oe:
                oe = oe['cofactor']

                if type(oe) is dict:
                    oe = [oe]

                for cofactor in oe:
                    cof = cofactor['Compound']['@frameid']
                    cofactor_set.add(cof)

        complex_df.at[i, 'enzyme_reaction'] = enz_rxn_set
        complex_df.at[i, 'cofactors'] = cofactor_set

    else:
        complex_df.at[i, 'enzyme_reaction'] = set()
        complex_df.at[i, 'cofactors'] = set()

In [ ]:
# save complex_df to parquet in a way that preserves dicts


In [ ]:
for col in ['enzyme_reaction', 'cofactors']:
    complex_df[col] = complex_df[col].apply(lambda x: list(x))

complex_df.to_parquet('notebooks/fbagd/data/raw_complexes.parquet', index=False, engine='fastparquet')
complex_df

## Create new column for monomer component stoichiometry

In [ ]:
test_complex_df = pd.read_parquet('notebooks/fbagd/data/raw_complexes.parquet')
test_complex_df

# convert cofactors and enzyme_reaction to str
for col in ['stoichiometry', 'enzyme_reaction', 'cofactors']:
    test_complex_df[col] = test_complex_df[col].astype(str)

    # interpret as str literal
    test_complex_df[col] = test_complex_df[col].apply(lambda x: ast.literal_eval(x))

test_complex_df

complex_names = complex_df['id'].tolist()
monomer_names = filter_protein_df['id'].tolist()

In [ ]:
def recursive_component_tree(current_component_name, complex_table, protein_table,
                             current_multiplier=1, component_list=None, parent=None, return_cofactors=False):
    """
    Recursively find all downstream components of a given complex.
    """

    complex_names = complex_table['id'].tolist()
    monomer_names = protein_table['id'].tolist()


    my_children = {}

    if component_list is None:
        component_list = []


    if current_component_name in complex_names:


        cplx_idx = complex_table.index[complex_table['id'] == current_component_name][0]
        stoichiometry = complex_table.at[cplx_idx, 'stoichiometry']

        for component_name, coefficient in stoichiometry.items():

            if coefficient < 0 and component_name != current_component_name:

                child_multiplier = abs(coefficient) * current_multiplier

                new_child = recursive_component_tree(component_name, complex_table, protein_table,
                                                     child_multiplier, component_list, current_component_name, return_cofactors)
                my_children = my_children | new_child


            elif coefficient > 0 and component_name == current_component_name:
                continue

            else:
                raise ValueError(f"key {k} and value {v} for complex {component_name} not processed properly.")

        component_list.append({'name': current_component_name,'parent': parent, 'children': list(my_children.keys()),
                               'multiplier': current_multiplier, })


    elif current_component_name in monomer_names:

        # TODO check if enzrxn
        if return_cofactors:
            protein_idx = protein_table.index[protein_table['id'] == current_component_name][0]

            protein_metals = protein_table.at[protein_idx, 'metal_features_processed']
            protein_other = protein_table.at[protein_idx, 'other_features_processed']

            table_cofactors = protein_metals | protein_other

            if len(table_cofactors) > 0:
                my_children = {f'APO-{current_component_name}': 1}
                component_list.append({'parent': current_component_name,
                                       'name': f'APO-{current_component_name}',
                                       'multiplier': current_multiplier,
                                       'children': None})

                for cofactor, cofactor_coefficient in table_cofactors.items():
                    if table_cofactors[cofactor] !=  None:
                        my_children[cofactor] = cofactor_coefficient
                        component_list.append({'parent': current_component_name,
                                               'name': cofactor,
                                               'multiplier': current_multiplier * cofactor_coefficient,
                                               'children': None})

        else:
            my_children = None

        component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': list(my_children.keys())})

    else:
        print(f"component {current_component_name} not found in complex or protein tables")

        return {}


    if parent is None:
        return {current_component_name: my_children}, component_list
    else:
        return {current_component_name: my_children}


In [ ]:
complex_tree_structure, nodes = recursive_component_tree('CPLX0-8167', complex_df, filter_protein_df)
pp.pprint(nodes)

In [ ]:
complex_df['monomer_component_stoichiometry'] = 0
complex_df['monomer_component_stoichiometry'] = complex_df['monomer_component_stoichiometry'].astype(object)

for i in range(len(complex_df.index)):
    complex_name = complex_df.loc[i, 'id']
    complex_tree_structure, nodes = recursive_component_tree(complex_name, complex_df, filter_protein_df)

    monomer_components = {node['name']: node['multiplier'] for node in nodes if node['children'] is None}

    complex_df.at[i, 'monomer_component_stoichiometry'] = monomer_components

In [ ]:
filter_complex_df = complex_df.loc[:, ["id", "monomer_component_stoichiometry", "cofactors"]]
filter_complex_df

In [ ]:
# filter_complex_df.to_parquet('notebooks/fbagd/data/processed_complexes.parquet', index=False)

# Get cofactor elemental composition and more

In [ ]:
# get set of all cofactors
all_metal_cofactors = set()
all_other_cofactors = set()

for i in range(len(filter_protein_df.index)):

    metal_cofactors = filter_protein_df.loc[i, 'metal_features_processed']
    other_cofactors = filter_protein_df.loc[i, 'other_features_processed']

    all_metal_cofactors = all_metal_cofactors | set(metal_cofactors.keys())
    all_other_cofactors = all_other_cofactors | set(other_cofactors.keys())

filter_cofactor_df = pd.DataFrame(columns=['id', 'elemental_composition'])
filter_cofactor_df['id'] = list(all_metal_cofactors | all_other_cofactors)

filter_cofactor_df['elemental_composition'] = 0
filter_cofactor_df['elemental_composition'] = filter_cofactor_df['elemental_composition'].astype(object)

# for each cofactor, get elemental composition
for i in range(len(filter_cofactor_df.index)):

    compound = filter_cofactor_df.loc[i, 'id']
    atom_dict = {}

    url_name = compound.replace('+', '%2b')

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{url_name}&detail=full'

    r = s.get(req_str)
    if r.status_code != 200:
        print(compound, r.status_code)
        filter_cofactor_df.at[i, 'elemental_composition'] = {}
        continue

    o = xmltodict.parse(r.content)['ptools-xml']

    atoms = o['Compound']['cml']['molecule']['atomArray']['atom']
    if type(atoms) is dict:
        atoms = [atoms]

    for atom in atoms:
        element = atom['@elementType']

        # either add new element or add to existing element
        if element not in atom_dict:
            atom_dict[element] = 1
        else:
            atom_dict[element] += 1

    filter_cofactor_df.at[i, 'elemental_composition'] = atom_dict

In [ ]:
filter_cofactor_df

In [ ]:
# save as parquet
# filter_cofactor_df.to_parquet('notebooks/fbagd/data/processed_cofactors.parquet', index=False)

# For each complex, look up components and check how many are missing from protein features.

In [ ]:
# filter_protein_df = pd.read_parquet('notebooks/fbagd/data/processed_proteins.parquet')
# filter_complex_df = pd.read_parquet('notebooks/fbagd/data/processed_complexes.parquet')
# filter_cofactor_df = pd.read_parquet('notebooks/fbagd/data/processed_cofactors.parquet')

In [ ]:
filter_complex_df

In [ ]:
 # for each complex, look up components and get cofactors for those components.
for i in range(len(filter_complex_df.index)):
    complex_id = filter_complex_df.loc[i, 'id']
    complex_components = filter_complex_df.loc[i, 'monomer_component_stoichiometry']
    complex_cofactors = filter_complex_df.loc[i, 'cofactors']

    current_combined_features = set()


    for component_name, component_count in complex_components.items():
        # get index of component in filter_protein_df
        component_index = filter_protein_df.index[filter_protein_df['id'] == component_name].tolist()[0]


        component_metals = filter_protein_df.at[component_index, 'metal_features_processed']
        component_other = filter_protein_df.at[component_index, 'other_features_processed']

        current_combined_features.update(set(component_metals.keys()))
        current_combined_features.update(set(component_other.keys()))

        # print(f'component_name: {component_name}, component_metals: {component_metals}, component_other: {component_other}, cofactors: {complex_cofactors}')

    # print difference between combined features and complex cofactors
    if len(complex_cofactors - current_combined_features) > 0:
        print(f'{complex_cofactors - current_combined_features}, {complex_id}, {current_combined_features}, {complex_cofactors}')


    filter_complex_df.at[i, 'cofactors'] = complex_cofactors

Seems neglible. Let's just use the protein features.

# Create matrices to get cofactor counts

In [ ]:
filter_protein_df

In [ ]:
# create protein name to index mapping
protein_name_to_index = {}
for i in range(len(filter_protein_df.index)):
    protein_name_to_index[filter_protein_df.at[i, 'id']] = i

# C matrix: complexes x proteins
C = np.zeros((len(filter_complex_df.index), len(filter_protein_df.index)))

for i in range(len(filter_complex_df.index)):

    complex_components = filter_complex_df.loc[i, 'monomer_component_stoichiometry']

    # TODO consider cofactors
    # complex_cofactors = filter_complex_df.loc[i, 'cofactors']


    for component_name, component_count in complex_components.items():
        if component_count is not None:             # side effect of parquet
            # get index of component in filter_protein_df
            component_index = protein_name_to_index[component_name]

            if filter_complex_df.at[i, 'id'] == 'APORNAP-CPLX':
                print(f'component_name: {component_name}, component_count: {component_count}, component_index: {component_index}')

            C[i, component_index] = component_count

# append an identity matrix to C
C = np.concatenate((C, np.identity(len(filter_protein_df.index))), axis=0)

C_names = list(filter_complex_df['id']) + list(filter_protein_df['id'])

In [ ]:
filter_complex_df.at[99, 'id']

In [ ]:
# create cofactor name to index mapping
cofactor_name_to_index = {}
for i in range(len(filter_cofactor_df.index)):
    cofactor_name_to_index[filter_cofactor_df.at[i, 'id']] = i

cofactor_names = list(filter_cofactor_df['id'])

# P matrix: proteins x cofactors
P = np.zeros((len(filter_protein_df.index), len(filter_cofactor_df.index)))

for i in range(len(filter_protein_df.index)):
    protein_metals = filter_protein_df.loc[i, 'metal_features_processed']
    protein_other = filter_protein_df.loc[i, 'other_features_processed']

    for metal, count in protein_metals.items():
        if count is not None:             # side effect of parquet
            cofactor_index = cofactor_name_to_index[metal]
            P[i, cofactor_index] = count

    for other, count in protein_other.items():
        if count is not None:             # side effect of parquet
            cofactor_index = cofactor_name_to_index[other]
            P[i, cofactor_index] = count


In [ ]:
P

In [ ]:
# create list of unique elements
unique_elements = set()
for i in range(len(filter_cofactor_df.index)):
    cofactor = filter_cofactor_df.at[i, 'elemental_composition']
    unique_elements.update(cofactor.keys())

unique_elements = list(unique_elements)

# create E matrix: cofactors x elements
E = np.zeros((len(filter_cofactor_df.index), len(unique_elements)))

for i in range(len(filter_cofactor_df.index)):
    cofactor = filter_cofactor_df.at[i, 'elemental_composition']

    for element, count in cofactor.items():
        if count is not None:             # side effect of parquet
            element_index = unique_elements.index(element)
            E[i, element_index] = count

In [ ]:
unique_elements

In [ ]:
C_to_E = C @ P @ E

In [ ]:
C_names[1577]

# Now ... add the counts >:o

In [ ]:
time = '50'
date = '2023-06-09'
experiment = 'fba-redux'
entry = f'{experiment}_{time}_{date}'
folder = f'out/fbagd/{entry}/'

In [ ]:
output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [ ]:
bulk

In [ ]:
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry


In [ ]:
initial_state = json.load(open('data/wcecoli_t0.json'))

bulk_ids = [item[0] for item in initial_state['bulk']]

bulk.columns = bulk_ids

In [ ]:
ecocyc_to_wcm_map = {}

# combined complex and protein names
protein_names = list(filter_protein_df['id']) + list(filter_complex_df['id'])

for name in protein_names:

    # find complex name in bulk_ids
    found = False

    try:
        idx = bulk_ids.index(name+'[c]')
        ecocyc_to_wcm_map[name] = name+'[c]'
        found = True
        # print(f'found {complex_name} at {idx}')

    except ValueError:
        # delete key
        found = False


    if found == False:

        for id in bulk_ids:
            if name+'[' in id and bulk.loc[:, id].sum() > 0:
                #print(f'found {name} in {id} with nonzero count')
                ecocyc_to_wcm_map[name] = id
                found = True
                break           # ensures preferring nonzero counts

            elif name+'[' in id:
                # print(f'found {name} in {id} with zero count')
                ecocyc_to_wcm_map[name] = id
                found = True

    if found == False:
        ecocyc_to_wcm_map[name] = '--TRANS-ACENAPHTHENE-12-DIOL[j]' # should be none
        print(f'could not find {name}')


In [ ]:
complex_wcm_names = [ecocyc_to_wcm_map[name] for name in C_names]

counts = bulk.loc[0, complex_wcm_names]

# Finally ... add the counts >:o

In [ ]:
factored_cofactor_elements = np.array(counts).reshape(-1,1) * C @ P @ E

In [ ]:
factored_cofactor_elements

In [ ]:
{unique_elements[i]: factored_cofactor_elements.sum(axis=0)[i] for i in range(len(unique_elements))}

In [ ]:
C_names[4443]

In [ ]:
C_names.index('APORNAP-CPLX')

In [ ]:
factored_cofactor_elements[99,:]

In [ ]:
# get filter_complex_df idx of APORNAP-CPLX
rnap_idx = filter_complex_df.index[filter_complex_df['id'] == 'APORNAP-CPLX'][0]

rnap_stoich = filter_complex_df.at[rnap_idx, 'monomer_component_stoichiometry']

for k,v in rnap_stoich.items():
    if v is not None:
        print(k,v)

In [ ]:
complex_tree_structure, nodes = recursive_component_tree('APORNAP-CPLX', complex_df, filter_protein_df)
pp.pprint(nodes)

In [ ]:
ecocyc_to_wcm_map['APORNAP-CPLX']

In [ ]:
bulk.loc[0, 'APORNAP-CPLX[c]']

In [ ]:
filter_complex_df[filter_complex_df['id'] == 'APORNAP-CPLX']

In [ ]:
rpoc_idx = filter_protein_df.index[filter_protein_df['id'] == 'RPOC-MONOMER'][0]

filter_protein_df.at[rpoc_idx, 'metal_features_processed']

In [ ]:
C[99,4108]

In [ ]:
protein_names.index('RPOC-MONOMER')

# Saving outputs to files compatible with Julia

In [ ]:
# save C, P and E to julia-compatible formats
np.savetxt('data/C_matrix.csv', C, delimiter=',')
np.savetxt('data/P_matrix.csv', P, delimiter=',')
np.savetxt('data/E_matrix.csv', E, delimiter=',')

# save names
complex_names = list(filter_complex_df['id'])
protein_names = list(filter_protein_df['id'])
cofactor_names = list(filter_cofactor_df['id'])
element_names = unique_elements

# write all names to single file with each list on a new line
with open('data/names.txt', 'w') as f:
    f.write('complexes:' + '\n'.join(complex_names))
    f.write('\n')
    f.write('proteins,' +'\n'.join(protein_names))
    f.write('\n')
    f.write('cofactors:' +'\n'.join(cofactor_names))
    f.write('\n')
    f.write('elements:' +'\n'.join(element_names))

# save counts of proteins and complexes
np.savetxt('data/counts.csv', np.array(counts), delimiter=',')

# save tree structures


In [ ]:
complex_tree_structure, nodes = recursive_component_tree('CPLX0-8167', complex_df, filter_protein_df, return_cofactors=True)
nodes

In [ ]:
np.array(counts)